In [ ]:
from tkinter import * 
from tkinter import filedialog 
import pandas as pd

list_filepath=[]

# load files via GUI
def openFile(): 
    window = Tk()
    button = Button(text = "Open", command= openFile)
    button.pack()
    
    filepath = filedialog.askopenfilename(initialdir="F:\\public\\PhD\\DataEvaluation\\DRIFT", 
                                                      title ='open spectra', 
                                                      filetypes = (("text files","*.txt"), ("all files", "*.*")))
    list_filepath.append(filepath)
    print(list_filepath)
    window.withdraw()
    return list_filepath[0]

# column name -> simplify
def rename_column():
    for df in dfs:
        df.columns = df.columns.str.replace(r'^.*?,\s*', '', regex=True)
    print(df.columns)

# column name -> ppm for the legend in subplots
def rename_column_for_legend():
    for dfx in dfs:
        dfx.columns = [dfx.columns[0]] + [f"{float(col.split()[0]):.0f} ppm" for col in dfx.columns[1:]]
    print(dfx.columns)
    
# if txt data has , inbetween: , -> .
def to_numeric_comma(col):
    if col.dtypes == 'object':
        # Replace comma with dot, remove spaces, then convert to float
        return pd.to_numeric(col.str.replace(',', '.').str.strip(), errors='coerce')
    else:
        return col  
    
    


In [ ]:
# Number of file to upload
file_n = int(input("Number of upload files: "))

In [ ]:
# load filepaths via fct:openFile 

while file_n > 0:
    file_n -= 1
    openFile()
    

In [ ]:
list_filepath
list_len = len(list_filepath)

In [ ]:
# create list ds with dataframes

dfs = [pd.read_csv(fp, sep='\t') for fp in list_filepath] # from each analyte gas filepath a pd df is created

In [ ]:
rename_column()

In [ ]:
rename_column_for_legend()

In [ ]:
# name each datarame in dfs to df1, df2, df3... etc.
for i, df in enumerate(dfs, start=1):
    globals()[f"df{i}"] = df

In [ ]:
df1

In [ ]:
# make a list of column names later for legend
column_name = df.columns.tolist()

In [ ]:
df1.iloc[0]

# functions for the plot

In [ ]:
def grd_color():
    x1 = df1[df1.columns[0]].values
    x2 = df2[df2.columns[0]].values
    
    colors_top = cm.Blues(np.linspace(0.3, 0.9, df1.shape[1]-1))    
    colors_bottom = cm.Reds(np.linspace(0.3, 0.9, df2.shape[1]-1))  
    
    for color, col in zip(colors_top, df1.columns[1:]):
        axes[0, 0].plot(x1, df1[col], color=color)
        axes[0, 0].set_xlim([2000, 1150])
        axes[0, 0].set_ylim([-0.02, 0.2])

    for color, col in zip(colors_top, df1.columns[1:]):
        axes[0, 1].plot(x1, df1[col], color=color)
        axes[0, 1].set_xlim([1150, 700])

    for color, col in zip(colors_bottom, df2.columns[1:]):
        axes[1, 0].plot(x2, df2[col], color=color)
        axes[1, 0].set_ylim([-0.01, 0.06])

    for color, col in zip(colors_bottom, df2.columns[1:]):
        axes[1, 1].plot(x2, df2[col], color=color)
    
    # create legend 
    legend_top = [Line2D([0], [0], color=color, lw=2, label=col) for color, col in zip(colors_top, df1.columns[1:])]
    legend_bottom = [Line2D([0], [0], color=color, lw=2, label=col) for color, col in zip(colors_bottom, df2.columns[1:])]

    axes[0, 1].legend(handles=legend_top, title='first as ref', loc='upper right', fontsize=14)
    axes[1, 1].legend(handles=legend_bottom, title='prev as ref', loc='upper right', fontsize=14)
    
def extra_legend():
    # labels for species
    labels = []
    colors = ['red', 'green', 'olive', 'orange', 'purple']
    custom_lines = [Line2D([0], [0], color=c, linestyle='--', lw=2, alpha=0.6) for c in colors]
    ax1.legend(custom_lines, labels, fontsize = 14) #, title='Assignments')
    
def add_scale_bar_tick(ax, xpos=0.05, ypos=0.05, text_offset=20, fontsize=14, color='k'):
    # Ensure ticks are drawn before measuring
    ax.figure.canvas.draw()
    yticks = ax.get_yticks()
    if len(yticks) < 2:
        return

    # bar length = y-axis tick interval
    tick_distance = yticks[1] - yticks[0]

    # convert relative positions to data coordinates
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    bar_x = x_min + xpos * (x_max - x_min)
    bar_y_start = y_min + ypos * (y_max - y_min)

    # add vertical bar
    ax.plot([bar_x, bar_x],
            [bar_y_start, bar_y_start + tick_distance],
            color=color, lw=2)

    # add label
    ax.text(bar_x + text_offset,
            bar_y_start + tick_distance / 2,
            f"{tick_distance:.2f}",
            fontsize=fontsize,
            va='center', ha='right', color=color)
    
def remove_ticks():
    ax1.tick_params(axis="both", length=0, labelbottom=False, labelleft=False)
    ax2.tick_params(axis="both", length=0, labelbottom=False, labelleft=False)
    ax3.tick_params(axis="y", length=0, labelbottom=True, labelleft=False)
    ax4.tick_params(axis="y", length=0, labelbottom=True, labelleft=False)



# axis and wavenumbers

In [ ]:
# define each wavenumbers depending on species
def plot_wavenumbers_numbers():
    # for species MUSTERSPECIES1
    MUSTERSPECIES1_wn = [] # here wavenumbers
    for wn in formate_wn:
        for ax in [ax1, ax3]:
            ax.axvline(x=wn, ymin = 0, ymax = 0.9, color='r', linestyle='--', alpha=0.5) 
            ax.text(wn+10, ax.get_ylim()[1], str(wn), rotation=90, color='r', verticalalignment='top')  

    # for species MUSTERSPECIES2
    MUSTERSPECIES2 = [] 
    for cb1_h in CB1_high_wavelengths:
        ax1.axvline(x=cb1_h, ymin = 0, ymax = 0.55, color='green', linestyle='--', alpha=0.5) 
        ax1.text(cb1_h+10, 0.121, str(cb1_h), rotation=90, color='green', verticalalignment='top')  
        ax3.axvline(x=cb1_h, ymin = 0, ymax = 0.68, color='green', linestyle='--', alpha=0.5) 
        ax3.text(cb1_h+10, 0.045, str(cb1_h), rotation=90, color='green', verticalalignment='top')  

  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import re
from matplotlib.lines import Line2D

# 2x2 sublplot
fig, axes = plt.subplots(nrows=2, ncols=2, figsize = (15,12), sharex='col', sharey = 'row', squeeze= True, dpi = 500)
plt.rcParams.update({
    'font.size': 15,           # base font size
    'axes.labelsize': 16,      # x and y axis labels
    'xtick.labelsize': 15,     # x-axis tick labels
    'ytick.labelsize': 15,     # y-axis tick labels
    })
plt.rcParams['font.family'] = 'Arial'

ax1 = df1.plot(x = df1.columns[0], y= df1.columns[1:], ax=axes[0,0], legend=False)
ax1.set_xlim([2000, 1150])
ax1.set_ylim([-0.02, 0.2])

ax2 = df1.plot(x = df1.columns[0], y= df1.columns[1:], ax=axes[0,1])
ax2.legend(fontsize=14) 

ax3 = df2.plot(x = df2.columns[0], y= df2.columns[1:], ax=axes[1,0], legend=False)
ax3.set_ylim([-0.01, 0.05])
ax3.set_xlabel('') 

ax4 = df2.plot(x = df2.columns[0], y= df2.columns[1:], ax=axes[1,1])
ax4.set_xlabel('') 
ax4.legend(fontsize=14) 

# Remove ticks
remove_ticks()

# Scalebar modification
add_scale_bar_tick(ax1, xpos=0.97, ypos=0.85, text_offset=10)
add_scale_bar_tick(ax3, xpos=0.97, ypos=0.96, text_offset=10)   
    
# Define blue and red gradients
grd_color()
fig.text(0.5, -0.015, 'Wavenumber / cm$^{-1}$', ha='center', fontsize=18)
fig.text(-0.005, 0.5, 'Absorbance / a.u.', va='center', rotation='vertical', fontsize=18)
plot_wavenumbers_numbers()
extra_legend()
fig.tight_layout()
fig.suptitle('title', fontsize=19, y=1.01)  

plt.show()
